# Assignment 2 — LD Score Regression (LDSC) with gwaslab
**Goal:** Estimate SNP-based (“chip”) heritability (h²) for sqrt(HDL) using LD Score Regression, then compute a 95% confidence interval and interpret it.

**Data sources**
- GWAS summary stats: `GERA-sqrtHDL.tsv.gz` (GWAS Catalog, GCST007140)
- LD reference panel: 1000 Genomes European (eur_w_ld_chr)
- HapMap3 SNP list: `w_hm3.snplist` (inside eur_w_ld_chr)

## Part 1: Check Environment + Files

In [9]:
import sys
import os
import gzip
from pathlib import Path

print("Python:", sys.version)
print("Working directory:", os.getcwd())

Python: 3.12.12 (main, Oct  9 2025, 11:07:00) [Clang 17.0.0 (clang-1700.4.4.1)]
Working directory: /Users/sarahmughal/Desktop/assign2_ldsc


## Part 2: Confirm Files Exist
This notebook expects the following folder structure:

- `tmp/GERA-sqrtHDL.tsv.gz`
- `tmp/eur_w_ld_chr/` (contains LD score reference files)
- `tmp/eur_w_ld_chr/w_hm3.snplist` (HapMap3 SNP list)

In [10]:
project_dir = Path(".")
tmp_dir = project_dir / "tmp"

gwas_file = tmp_dir / "GERA-sqrtHDL.tsv.gz"
hm3_list = tmp_dir / "eur_w_ld_chr" / "w_hm3.snplist"
ref_dir = tmp_dir / "eur_w_ld_chr"

print("GWAS file exists:", gwas_file.exists(), gwas_file)
print("HapMap3 list exists:", hm3_list.exists(), hm3_list)
print("Reference dir exists:", ref_dir.exists(), ref_dir)

GWAS file exists: True tmp/GERA-sqrtHDL.tsv.gz
HapMap3 list exists: True tmp/eur_w_ld_chr/w_hm3.snplist
Reference dir exists: True tmp/eur_w_ld_chr


In [18]:
# First few lines of the GWAS summary stats file
with gzip.open(gwas_file, "rt") as f:
    for i in range(5):
        print(f.readline().rstrip())

SNP_ID	chromosome	position	Allele 1	Allele 2	Effect allele (EA)	Effect allele frequency (EAF)	Sample size	Estimate Effect	P value
rs10458597	1	564621	T	C	T	0.116	6855	-0.00873	0.54
rs8179414	1	565400	T	C	T	0.0503	6855	-0.0261	0.21
rs9701055	1	565433	T	C	T	0.0169	91277	-0.0119	0.31
rs189147642	1	721757	T	A	T	0.963	76627	-0.00827	0.47


## Part 3: Filter GWAS summary stats to HapMap3 SNPs (memory efficient)

Why do this?
- LDSC is commonly run on HapMap3 SNPs to reduce noise and keep runtime/memory manageable.
- We load the HapMap3 SNP IDs into a `set`, then stream through the gzipped GWAS file line-by-line and write out only matching SNPs.

In [12]:
filtered_file = tmp_dir / "GERA-sqrtHDL-hm3.tsv.gz"

# Load HapMap3 SNP IDs into a set
with open(hm3_list, "r") as f:
    hm3_snps = {line.split("\t")[0] for line in f if line.strip()}

print(f"Loaded {len(hm3_snps):,} HapMap3 SNP IDs.")

# Stream input and write filtered output
count_in = 0
count_out = 0

with gzip.open(gwas_file, "rt") as f_in, gzip.open(filtered_file, "wt") as f_out:
    header = f_in.readline()
    f_out.write(header)

    for line in f_in:
        count_in += 1
        snp_id = line.split("\t", 1)[0]  # SNP_ID is the first column
        if snp_id in hm3_snps:
            f_out.write(line)
            count_out += 1

print(f"Read {count_in:,} SNPs from the original file.")
print(f"Wrote {count_out:,} HapMap3 SNPs to: {filtered_file}")


Loaded 1,217,312 HapMap3 SNP IDs.
Read 20,325,718 SNPs from the original file.
Wrote 1,210,412 HapMap3 SNPs to: tmp/GERA-sqrtHDL-hm3.tsv.gz


In [ ]:
# First few lines of the filtered file
with gzip.open(filtered_file, "rt") as f:
    for i in range(5):
        print(f.readline().rstrip())

SNP_ID	chromosome	position	Allele 1	Allele 2	Effect allele (EA)	Effect allele frequency (EAF)	Sample size	Estimate Effect	P value
rs3094315	1	752566	G	A	G	0.164	94235	-0.00505	0.32
rs3131972	1	752721	A	G	A	0.198	94674	-0.000876	0.84
rs3131969	1	754182	A	G	A	0.172	94674	-0.0013	0.78
rs1048488	1	760912	C	T	C	0.314	10753	0.0131	0.28


## Part 4: Run LD Score Regression (LDSC) using gwaslab

We load the filtered summary stats (HapMap3 SNPs) and run `estimate_h2_by_ldsc()`.

Notes:
- The summary stats are build hg19 / b37 (gwaslab uses `build="19"` for hg19).
- `ref_ld_chr` and `w_ld_chr` both point to the same reference directory for this dataset.


In [ ]:
import gwaslab as gl

print("gwaslab version loaded successfully.")

ss = gl.Sumstats(
    str(filtered_file),
    snpid="SNP_ID",
    chrom="chromosome",
    pos="position",
    ea="Effect allele (EA)",
    nea="Allele 2",
    n="Sample size",
    p="P value",
    beta="Estimate Effect",
    build="19",
)

# Basic formatting check
ss.basic_check()

gwaslab version loaded successfully.
2026/01/19 12:02:40 GWASLab v4.0.4 https://cloufield.github.io/gwaslab/
2026/01/19 12:02:40 (C) 2022-2026, Yunye He, Kamatani Lab, GPL-3.0 license, gwaslab@gmail.com
2026/01/19 12:02:40 Python version: 3.12.12 (main, Oct  9 2025, 11:07:00) [Clang 17.0.0 (clang-1700.4.4.1)]
2026/01/19 12:02:40 Start to initialize gl.Sumstats from file :tmp/GERA-sqrtHDL-hm3.tsv.gz
2026/01/19 12:02:41  -Reading columns          : SNP_ID,chromosome,Estimate Effect,P value,Allele 2,Sample size,Effect allele (EA),position
2026/01/19 12:02:41  -Renaming columns to      : SNPID,CHR,BETA,P,NEA,N,EA,POS
2026/01/19 12:02:41  -Current Dataframe shape : 1210412  x  8
2026/01/19 12:02:41  -Initiating a status column: STATUS ...
2026/01/19 12:02:41  -Genomic coordinates are based on GRCh37/hg19...
2026/01/19 12:02:41 Start to reorder the columns ...(v4.0.4)
2026/01/19 12:02:41  -Reordering columns to    : SNPID,CHR,POS,EA,NEA,STATUS,BETA,P,N
2026/01/19 12:02:41 Finished reordering

,SNPID,CHR,POS,EA,NEA,STATUS,BETA,P,N
0,rs3094315,1,752566,G,A,1980099,-0.005050,0.3200,94235
1,rs3131972,1,752721,A,G,1980099,-0.000876,0.8400,94674
2,rs3131969,1,754182,A,G,1980099,-0.001300,0.7800,94674
3,rs1048488,1,760912,C,T,1980099,0.013100,0.2800,10753
4,rs3115850,1,761147,T,C,1980099,0.013200,0.2700,10753
...,...,...,...,...,...,...,...,...,...
1210407,rs3810648,22,51175626,A,G,1980099,0.012300,0.0960,87819
1210408,rs2285395,22,51178090,G,A,1980099,0.022500,0.0076,87819
1210409,rs3865766,22,51186228,C,T,1980099,-0.001900,0.8800,7795
1210410,rs3888396,22,51211392,T,C,1980099,-0.004100,0.7100,14650


In [ ]:
results = ss.estimate_h2_by_ldsc(
    ref_ld_chr=str(ref_dir) + "/", w_ld_chr=str(ref_dir) + "/", munge=True
)

# The main output is stored here:
ss.ldsc_h2

2026/01/19 12:02:43  -Genomic coordinates are based on GRCh37/hg19...
2026/01/19 12:02:43 Start to extract HapMap3 SNPs ...(v4.0.4)
2026/01/19 12:02:43  -Loading Hapmap3 variants from built-in datasets...
2026/01/19 12:02:45  -Since rsID not in sumstats, CHR:POS( build 19) will be used for matching...
2026/01/19 12:02:46  -Checking if alleles are same...
2026/01/19 12:02:47  -Variants with matched alleles: 1213751
2026/01/19 12:02:47  -Raw input contains 1213751 Hapmap3 variants based on CHR:POS...
2026/01/19 12:02:47 Finished extracting HapMap3 SNPs.
2026/01/19 12:02:47 Start to run LD score regression ...(v4.0.4)
2026/01/19 12:02:47  -Current Dataframe shape : 1213751 x 10 ; Memory usage: 85.66 MB
2026/01/19 12:02:47 Start to munge sumstats.
2026/01/19 12:02:47  -Renamed columns: {'EA': 'A1', 'NEA': 'A2', 'rsID': 'SNP'}
2026/01/19 12:02:47  -WARNING: 6658 SNPs had P outside of (0,1]. The P column may be mislabeled.
2026/01/19 12:02:47  -After P-value filtering: 1207093 SNPs remain
20

,h2_obs,h2_se,Lambda_gc,Mean_chi2,Intercept,Intercept_se,Ratio,Ratio_se,Catagories
0,0.2130941,0.02382769,1.25445562,1.46028661,1.04590234,0.01174202,0.09972557,0.02551024,NA


## Part 5: 95% Confidence Interval For h²

We compute a 95% CI using:

\[
h^2 \pm 1.96 \times SE
\]

Then we interpret what this means in plain language.


In [20]:
ldsc_out = ss.ldsc_h2
print("Type of ss.ldsc_h2:", type(ldsc_out))
print(ldsc_out)

Type of ss.ldsc_h2: <class 'pandas.core.frame.DataFrame'>
      h2_obs       h2_se   Lambda_gc   Mean_chi2   Intercept Intercept_se  \
0  0.2130941  0.02382769  1.25445562  1.46028661  1.04590234   0.01174202   

        Ratio    Ratio_se Catagories  
0  0.09972557  0.02551024         NA  


In [ ]:
import numpy as np
import pandas as pd

ld = ss.ldsc_h2.copy()

# If it's not a DataFrame, stop and we’ll handle it differently
if not isinstance(ld, pd.DataFrame):
    raise TypeError(f"Expected ss.ldsc_h2 to be a DataFrame, got: {type(ld)}")

# normalize column names for searching
col_map = {
    c: c.lower().replace(" ", "").replace("-", "").replace("_", "") for c in ld.columns
}

# candidate patterns
h2_candidates = [
    c
    for c, norm in col_map.items()
    if norm in ("h2", "h2obs", "snph2", "totalh2", "h2total", "h2observed")
]
se_candidates = [
    c
    for c, norm in col_map.items()
    if norm in ("se", "seh2", "h2se", "std", "stderr", "standarderror")
]

print("Possible h2 columns:", h2_candidates)
print("Possible SE columns:", se_candidates)

# pick the first reasonable ones
h2_col = h2_candidates[0] if h2_candidates else None
se_col = se_candidates[0] if se_candidates else None

if h2_col is None or se_col is None:
    raise KeyError(
        "Could not automatically find h2 and SE columns. Paste the 'Columns:' output and I'll tell you exactly which ones to use."
    )

# if there are multiple rows, grab the first row (usually correct for single-trait LDSC)
h2 = float(ld.iloc[0][h2_col])
se = float(ld.iloc[0][se_col])

ci_low = h2 - 1.96 * se
ci_high = h2 + 1.96 * se

print(f"Using columns: h2='{h2_col}', SE='{se_col}'")
print("h2 =", h2)
print("SE =", se)
print("95% CI =", (ci_low, ci_high))

Possible h2 columns: ['h2_obs']
Possible SE columns: ['h2_se']
Using columns: h2='h2_obs', SE='h2_se'
h2 = 0.2130941
SE = 0.02382769
95% CI = (0.16639182760000001, 0.2597963724)


## Conclusion: Genome-Wide SNP-based Heritability

Genome-wide SNP-based heritability for square-root–transformed HDL cholesterol was estimated using LD Score Regression implemented in the `gwaslab` package, using European ancestry LD reference panels from the 1000 Genomes Project and restricting analyses to HapMap3 SNPs. The estimated heritability was **h² = 0.21**, with a standard error of **0.024**. A 95% confidence interval was calculated as \( h^2 \pm 1.96 \times SE \), resulting in a confidence interval of **(0.17, 0.26)**. This result suggests that about **21%** of the variation in square-root–transformed HDL cholesterol can be explained by common genetic variants captured by GWAS arrays. The remaining variation is likely due to environmental factors, measurement error, and genetic effects not well captured by common SNPs, such as rare variants. Because this estimate reflects SNP-based (chip) heritability, it is expected to be lower than heritability estimates from twin or family-based studies.
